In [ ]:
# Import and create a new SQLContext 
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
# Read the country CSV file into an RDD.
country_lines = sc.textFile('file:///home/cloudera/Downloads/big-data-3/final-project/country-list.csv')

In [ ]:
# Convert each line into a pair of words
country_lines = country_line.map(lambda line: tuple(line.split(",")))

In [ ]:
country_lines.take(8)

In [ ]:
# Convert each pair of words into a tuple


In [ ]:
# Create the DataFrame, look at schema and contents
countryDF = sqlContext.createDataFrame(country_tuples, ["country", "code"])
countryDF.printSchema()
countryDF.take(3)

In [ ]:
# Read tweets CSV file into RDD of lines
results_lines = sc.textFile('file:///home/cloudera/Downloads/big-data-3/mongodb/results.csv')
results_lines.take(5)

In [ ]:
results = results_lines.filter(lambda x: x is not u'')
results.take(5)

In [ ]:
res_words = results.flatMap(lambda x: x.split(" "))
res_tupls = res_words.map(lambda x: (x, 1))
res_count = res_tupls.reduceByKey(lambda x,y: (x+y))

In [ ]:
res_count.take(5)

In [ ]:
tweetDF = sqlContext.createDataFrame(res_count, ['word', 'count'])
tweetDF.printSchema()
tweetDF.take(5)

In [ ]:
# Question 2: number of countries mentioned in tweets.
from pyspark.sql.functions import sum, col
joinedDF = tweetDF.alias('a').join(countryDF.alias('b'), col('a.word') == col('b.country')).select(
    col('a.count'),col('b.country'),col('b.count'))
joinedDF.printSchema()
joinedDF.take(5)

In [ ]:
# Q1: number of distinct countries mentioned
joinedDF.count(), joinedDF.select('code').distinct().count()

In [ ]:
# Q2: number of countries mentioned in tweets
from pyspark.sql.functions import sum
joinedDF.agg(sum('count')).first()

In [ ]:
# Table 1: top three countries and their counts.
from pyspark.sql.functions import desc
sortedDF = joinedDF.sort(desc("count"))
sortedDF.show(5)

In [ ]:
# Table 2: counts for Wales, Iceland, and Japan.
tbl2DF = joinedDF.where((col("country")=="Wales")|(col("country")=="Iceland")|(col("country")=="Japan"))
tbl2DF.show()

In [ ]:
tbl3DF = joinedDF.where((col("country")=="Wales")|(col("country")=="Kenya")|(col("country")=="Netherlands"))
tbl3DF.show()

In [ ]:
from pyspark.sql.functions import avg
joinedDF.agg(avg('count')).first()